# 0.0. IMPORTS

In [1]:
import re
import datetime
import requests
import pandas as pd

from bs4 import BeautifulSoup

# 1.0. COLETA DE DADOS

    - Aqui eu vou tentar fazer um scrapy geral usando todas as paginas, pegando minhas informações principais
    - Depois vou entrar no detalhe de cada produto pra pegar as informações adicionais 
    - Vou fazer dois dataframes e usar meu titulo pra mergear os dois 

## 1.1. Api Request

In [2]:
url = 'https://books.toscrape.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get(url, headers = headers)

soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
# Total de produtos no site
total_products = soup.find_all('form', class_="form-horizontal")[0].get_text().split('\n') # 1000

total_products = list(filter(None, total_products))[0]
total_products = total_products[0:4]

# Total de paginas no site
pages_number = round(int(total_products) / 20) # 50 

## 1.2.TODAS AS PAGINAS

In [4]:
aux_details = []
df_aux = pd.DataFrame(columns = ['title', 'price', 'star_rate'])


for i in range( 1, pages_number+1 ):
    
    # pagina a pagina até a ultima do catalogo
    url = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'
    
    # api request
    page = requests.get( url, headers=headers )
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    all_products = soup.find( 'ol', class_='row' ) # vitrine
    product_list = all_products.find_all('a', href=True) # link de cada pagina
    
    # Pegando o nome dos titulos de cada livro 
    list_title = soup.find_all('a', href=True)

    title = [i.get('title') for i in list_title]

    # Fazendo a limpeza dos "None's" que vieram junto
    title = list(filter(None, title))
    
    # Pegando o preço de cada livro
    list_price = soup.find_all('p', class_='price_color')

    prices = [i.get_text() for i in list_price]
    
    # Avaliações dos clientes
    evaluation_list = soup.find_all('p', class_='star-rating')

    evaluation = [i.get('class')[1] for i in evaluation_list]
    
    # Criando um dataframe
    data = pd.DataFrame( [title, prices, evaluation]).T
    data.columns = ['title', 'price', 'star_rate']
    
    data['scrapy_datetime'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    df_aux = pd.concat([df_aux, data], axis=0)
    
    
    for i in range( 1, 40, 2 ):
        aux_details.append( product_list[i]['href'] )
        

## 1.3. DETALHE DO PRODUTO

In [5]:
aux_title = []
aux_availability = []
aux_reviews = []
aux_category = []

for i in range(len(aux_details)):
    url = 'https://books.toscrape.com/catalogue/' + aux_details[i]
    
    page = requests.get(url, headers = headers)

    soup = BeautifulSoup(page.text, 'html.parser')
    
    title = soup.find_all('h1')[0].get_text()
    aux_title.append(title)
    
    # Estoque de produtos
    product_availability = soup.find_all('p', class_= 'instock availability')[0].get_text().split('\n')
    product_availability = product_availability[3]
    
    aux_availability.append(product_availability)
    
    # Avaliação do produto
    product_review = soup.find_all('td')[6].get_text()
    
    aux_reviews.append(product_review)
    
    # Categoria do livro
    book_category = soup.find_all('a', href=True)[3].get_text()
    
    aux_category.append(book_category)

df_details = pd.DataFrame([aux_title, aux_category, aux_availability, aux_reviews]).T
df_details.columns = ['title', 'book_category', 'stock', 'book_review']

In [6]:
data_final = pd.merge(df_details, df_aux, on = 'title', how = 'left')

In [7]:
data_final = data_final[['title', 'book_category', 'price', 'star_rate', 'stock']].copy()

#  2.0. DESCRIÇÃO DOS DADOS

In [8]:
data = data_final.copy()

data.head()

,title,book_category,price,star_rate,stock
0,A Light in the Attic,Poetry,Â£51.77,Three,In stock (22 available)
1,Tipping the Velvet,Historical Fiction,Â£53.74,One,In stock (20 available)
2,Soumission,Fiction,Â£50.10,One,In stock (20 available)
3,Sharp Objects,Mystery,Â£47.82,Four,In stock (20 available)
4,Sapiens: A Brief History of Humankind,History,Â£54.23,Five,In stock (20 available)


## 2.1. Dimensões do Dataframe

In [9]:
print(f'Quantidade de linhas do meu dataframe: {data.shape[0]}')
print(f'Quantidade de colunas do meu dataframe: {data.shape[1]}')

Quantidade de linhas do meu dataframe: 1002
Quantidade de colunas do meu dataframe: 5


## 2.2. Tipos de dados

In [10]:
data.dtypes

title            object
book_category    object
price            object
star_rate        object
stock            object
dtype: object

## 2.3. Checando NA's

In [11]:
data.isnull().sum()

title            0
book_category    0
price            0
star_rate        0
stock            0
dtype: int64

## 2.4. Transformando os dados


In [12]:
# Colocando os nomes dos meus titulos em minusculo
data['title'] = data['title'].apply(lambda x: x.lower() if pd.notnull(x) else x) 

# Colocando as categorias dos livros em minusculo
data['book_category'] = data['book_category'].apply(lambda x: x.lower() if pd.notnull(x) else x) 

# Retirando os caracteres 'Â£' do meu preço
data['price'] = data['price'].apply(lambda x: x.replace('Â£', '')).astype(float)

# star rate para numerico
data['star_rate'] = data['star_rate'].apply(lambda x: 1 if x == 'One' else
                                                    2 if x == 'Two' else
                                                    3 if x == 'Three' else
                                                    4 if x == 'Four' else 5).astype('int64')

# Pegando somente o numero de avaliações
data['stock'] = data['stock'].apply(lambda x: re.search('\d?\d', x).group(0) if pd.notnull(x) else x).astype('int64')

# Removendo duplicidades do meu dataframe
data = data.drop_duplicates(keep='last')

## 2.5. Resultado final

In [13]:
data.head()

,title,book_category,price,star_rate,stock
0,a light in the attic,poetry,51.77,3,22
1,tipping the velvet,historical fiction,53.74,1,20
2,soumission,fiction,50.10,1,20
3,sharp objects,mystery,47.82,4,20
4,sapiens: a brief history of humankind,history,54.23,5,20
